## Quantize SD 3.5 Medium

In [ ]:
from diffusers import BitsAndBytesConfig, SD3Transformer2DModel
import torch

# Define the model ID and cache directory
model_cache_dir = "sd"
model_id = "stabilityai/stable-diffusion-3.5-medium"

# Quantization configuration (nf4)
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Load the model with the quantization config
model_nf4 = SD3Transformer2DModel.from_pretrained(
    model_id,
    subfolder="transformer",
    quantization_config=nf4_config,
    torch_dtype=torch.float16,
    cache_dir=model_cache_dir
)

# Save the quantized model to the "quantized_model" directory
quantized_model_dir = "quantized_model/sd3.5med"
model_nf4.save_pretrained(quantized_model_dir)

print(f"Quantized model saved in {quantized_model_dir}")

## RUN Quantized SD 3.5 Medium

In [ ]:
from diffusers import StableDiffusion3Pipeline, SD3Transformer2DModel
import torch

# Define the model ID and cache directory
model_cache_dir = "sd"
model_id = "stabilityai/stable-diffusion-3.5-large"

# Load the quantized model from the 'quantized_model' directory
quantized_model_dir = "quantized_model/sd3.5med"
model_nf4 = SD3Transformer2DModel.from_pretrained(
    quantized_model_dir,
    torch_dtype=torch.float16, 
    cache_dir=model_cache_dir
)

# Load the pipeline using the quantized model
pipeline = StableDiffusion3Pipeline.from_pretrained(
    model_id,
    transformer=model_nf4,
    torch_dtype=torch.float16,
    cache_dir=model_cache_dir
)
pipeline.enable_model_cpu_offload()
# Define the prompt for image generation

prompt = "anime red hair girl, with yellow eyes, in garden"
# Generate the image
image = pipeline(
    prompt=prompt,
    num_inference_steps=40,
    guidance_scale=4.5,
).images[0]

# Save the generated image
image.save("generated_image_from_quantized_model_sd3.5med.png")
print("Image generated and saved as 'generated_image_from_quantized_model_sd3.5med.png'")
image.show()